In [ ]:
import PyPDF2

# Path to the PDF file
pdf_path = 'sssv1.pdf'

newfile = open("txtfile.txt","w")

# Open the PDF file
with open(pdf_path, 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    num_pages = len(reader.pages)
    
    # Iterate through all pages and extract text
    for page_num in range(num_pages):
        page = reader.pages[page_num]
        text = page.extract_text()
        newfile.write(text)
        #print(text)
        #print("-" * 40)
newfile.close()        

: 

In [1]:
text = open("txtfile.txt").read()

In [2]:
len(text)

370044

In [3]:
print(text[:1000])

Sri Sathya Sai Speaks, Vol 1 (1953 - 60)
Divine Discourses of Bhagawan Sri Sathya Sai BabaIndex Of Discourses
1.   Have You Heard Our Baba Speak? ....................................................................... 2
1.   Worship In The Mind .............................................................................................. 7
2.   Total Surrender ...................................................................................................... 13
3.   God As Guide ......................................................................................................... 20
4.   Divine Life .............................................................................................................. 25
5.   Meditation On The Lord's Name And Form ...................................................... 27
6.   An Attitude Of Challenge ...................................................................................... 33
7.   Courage ......................................

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [5]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[ch] for ch in s]
decode = lambda s: ''.join([itos[ch] for ch in s])

In [6]:
decode(encode("sairam"))

'sairam'

In [7]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
enc.encode("sairam")

/Users/suriya/personal/swami-bot/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[82, 958, 321]

In [8]:
enc.decode([82,958,321])

'sairam'

In [14]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [15]:
block_size=8
train_data[:block_size+1]

tensor([42, 67, 58,  1, 42, 50, 69, 57, 74])

In [29]:
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=="train" else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [62]:
from torch import nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss=None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next),dim=1)
        
        return idx



model = BigramLanguageModel(vocab_size=vocab_size)
x, y = get_batch("train")
logits = model.forward(x,y)

In [63]:
idx = torch.zeros((1,1),dtype=torch.long)
decode(model.generate(idx,max_new_tokens=100)[0].tolist())

'\n"i¦-oD vKâx" ?IHl/fwQJa(\'ceWFAKp50NF\nSzS7hh-IUeNP/qTlqZmK\nb;EyN1uBP[EoBvb10TL ba I/¦j9-bGy1p(8nw9:.!'

In [64]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)


In [67]:
batch_size = 32
for steps in range(10000):
    x,y = get_batch("train")
    logits, loss = model.forward(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps%100==0:
        print (steps, loss)


0 tensor(3.2938, grad_fn=<NllLossBackward0>)
100 tensor(3.1281, grad_fn=<NllLossBackward0>)
200 tensor(3.1281, grad_fn=<NllLossBackward0>)
300 tensor(2.9321, grad_fn=<NllLossBackward0>)
400 tensor(3.0231, grad_fn=<NllLossBackward0>)
500 tensor(3.0287, grad_fn=<NllLossBackward0>)
600 tensor(2.9190, grad_fn=<NllLossBackward0>)
700 tensor(2.8607, grad_fn=<NllLossBackward0>)
800 tensor(2.9269, grad_fn=<NllLossBackward0>)
900 tensor(2.8112, grad_fn=<NllLossBackward0>)
1000 tensor(2.8645, grad_fn=<NllLossBackward0>)
1100 tensor(2.6287, grad_fn=<NllLossBackward0>)
1200 tensor(2.7942, grad_fn=<NllLossBackward0>)
1300 tensor(2.7157, grad_fn=<NllLossBackward0>)
1400 tensor(2.9238, grad_fn=<NllLossBackward0>)
1500 tensor(2.7871, grad_fn=<NllLossBackward0>)
1600 tensor(2.8025, grad_fn=<NllLossBackward0>)
1700 tensor(2.5992, grad_fn=<NllLossBackward0>)
1800 tensor(2.6993, grad_fn=<NllLossBackward0>)
1900 tensor(2.7128, grad_fn=<NllLossBackward0>)
2000 tensor(2.6359, grad_fn=<NllLossBackward0>)
2100

In [76]:
idx = torch.zeros((1,1),dtype=torch.long)
print (decode(model.generate(idx,max_new_tokens=500)[0].tolist()))


I..
8Sath lldaseat wisherive thinorre Gof bea stsexenteveande wher tean wn.se indn ste vingo cha thowe..sid 2.
Sa trelethife a angakedarext is, "RUna
ste ny urst Pet alas mir s. aithemeally
f arale werisisochencthe Plin totht, Dothev) I wasthatreshy I tive be de hire'istoomo n,
f hat imy ne d tu tmicong ind'same e- on Gio adale.... Cure  rnan an d atheared ary ay o a f ope woripouthingr - vit 
ictaksake Ave secou tuas
bus he ge,
theh 1660 ghantipe inda aiveed; Whmene or th  the o sisiranoranotps
